In [1]:
import os
import glob

import DESutils
import GPRutils
import vK2KGPR
import plotGPR
import vK2KGPR
import vonkarmanFT as vk

import numpy as np
import astropy.units as u
import astropy.table as tb
from astropy.time import Time
import matplotlib.pyplot as plt
plt.style.use('~/GitHub/custom-matplotlib/custom.mplstyle')
import astropy.stats as stats
from scipy.signal import savgol_filter

from IPython import embed
from importlib import reload
import warnings
warnings.filterwarnings("ignore")

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
FITSfiles = sorted(glob.glob(f"/home/fortino/GPRSolutions/zone134/GPR.??????.?RC.fits"))
FITSfiles += sorted(glob.glob(f"/home/fortino/GPRSolutions/zone133/GPR.??????.?RC.fits"))
FITSfiles += sorted(glob.glob(f"/home/fortino/GPRSolutions/zone132/GPR.??????.?RC.fits"))
FITSfiles += sorted(glob.glob(f"/home/fortino/GPRSolutions/eris/GPR.??????.?RC.fits"))
print(len(FITSfiles))

343


In [3]:
%%time
AP = plotGPR.AggregatePlots(FITSfiles)
AP.calcVals()

CPU times: user 3min 5s, sys: 8.69 s, total: 3min 14s
Wall time: 3min 15s


In [4]:
def printStat(arr):
    mean = np.mean(arr)
    std = np.std(arr)
    perc25, perc50, perc75 = np.percentile(arr, (25, 50, 75))
    print(f"{mean:.4f} ± {std:.4f} ({perc25:.4f}, {perc50:.4f}, {perc75:.4f})")

In [5]:
xi0_raw = []
xi0_GPR_fC = []
xi0_GPR = []

RMS_raw = []
RMS_GPR_fC = []
RMS_GPR = []

red = []
red_fC = []

for band in ["r", "i", "z"]:
    xi0_raw.extend(AP.xi0_raw[band])
    xi0_GPR_fC.extend(AP.xi0_GPR_fC[band])
    xi0_GPR.extend(AP.xi0_GPR[band])
    
    RMS_raw.extend(AP.RMS_raw[band])
    RMS_GPR_fC.extend(AP.RMS_GPR_fC[band])
    RMS_GPR.extend(AP.RMS_GPR[band])

    red.extend(AP.red[band])
    red_fC.extend(AP.red_fC[band])

In [6]:
printStat(xi0_raw)
printStat(xi0_GPR_fC)
printStat(xi0_GPR)

125.0111 ± 126.0278 (52.7755, 79.0094, 145.4569)
19.4813 ± 16.0442 (8.6543, 15.1201, 24.9066)
11.9657 ± 11.3459 (5.4507, 8.2430, 14.8901)


In [7]:
printStat(RMS_raw)
printStat(RMS_GPR_fC)
printStat(RMS_GPR)

7.2637 ± 3.1216 (5.1369, 6.2852, 8.5280)
2.9184 ± 1.1063 (2.0802, 2.7495, 3.5289)
2.2650 ± 0.9234 (1.6509, 2.0301, 2.7286)


In [8]:
printStat(red)
printStat(red_fC)

12.0173 ± 7.0324 (7.1628, 10.2450, 15.1713)
6.6940 ± 3.2299 (4.5693, 5.6363, 7.9260)


In [12]:
starDensity = []
for band in plotGPR.DES_PASSBANDS:
    starDensity.extend(AP.starDensity[band])

printStat(u.Quantity(starDensity).value)

N = (u.Quantity(starDensity).value*3*u.deg**2 * u.arcmin**-2).decompose().value
printStat(N)
np.mean(N)*0.2, np.mean(N)*0.8

0.9469 ± 0.3353 (0.8185, 1.0064, 1.2396)
10226.0408 ± 3620.8619 (8839.5000, 10869.0000, 13387.5000)


(2045.2081632653062, 8180.832653061225)

In [13]:
nStepsGP = []
for band in plotGPR.DES_PASSBANDS:
    nStepsGP.extend(AP.nStepsGP[band])
printStat(nStepsGP)

106.1137 ± 60.6904 (61.5000, 87.0000, 133.0000)


In [14]:
nSteps1 = []
for band in plotGPR.DES_PASSBANDS:
    nSteps1.extend(AP.nSteps1[band])
printStat(nSteps1)

73.5160 ± 48.7065 (40.0000, 56.0000, 96.0000)


In [15]:
nSteps2 = []
for band in plotGPR.DES_PASSBANDS:
    nSteps2.extend(AP.nSteps2[band])
printStat(nSteps2)

32.5977 ± 30.2143 (15.0000, 24.0000, 33.0000)


In [14]:
len(AP.expNums["u"]), len(AP.expNums["g"]), len(AP.expNums["r"]), len(AP.expNums["i"]), len(AP.expNums["z"]), len(AP.expNums["Y"])

(0, 76, 70, 64, 72, 61)

In [15]:
len(FITSfiles)

343

In [17]:
s = []
for band in plotGPR.DES_PASSBANDS:
    s.extend(AP.nStepsfC[band])
np.mean(s), np.percentile(s, (25, 50, 75)), np.std(s)

(119.20116618075802, array([ 91., 150., 150.]), 38.66892890655189)

In [ ]:
# for band in plotGPR.DES_PASSBANDS:
# #     assert band in DES_PASSBANDS
# #     if len(self.expNums[band]) == 0:
# #         return
    
#     plt.hist(
#         AP.RMS_raw[band],
#         bins=50, range=(0, 25),
# #         histtype="step",
#         color=plotGPR.DES_COLORS[band],
#         label=f"{band}")

# plt.legend()
# plt.show()

# for band in plotGPR.DES_PASSBANDS:
# #     assert band in DES_PASSBANDS
# #     if len(self.expNums[band]) == 0:
# #         return
    
#     plt.hist(
#         AP.RMS_GPR[band],
#         bins=50, range=(0, 25),
# #         histtype="step",
#         color=plotGPR.DES_COLORS[band],
#         label=f"{band}")

# plt.legend()
# plt.show()

for band in plotGPR.DES_PASSBANDS:
    
    # If there are no exposures for that band in AP, then skip it.
    if len(AP.expNums[band]) == 0:
        continue
    
    densities = u.Quantity(AP.starDensity[band]).value
    plt.hist(densities, color=plotGPR.DES_COLORS[band], label=band)
    
plt.legend()
plt.show()

In [ ]:
times = np.array([])
GPtimes = np.array([])
nOpt1 = np.array([])
nOpt2 = np.array([])
nGP = np.array([])
for outfile in outfiles:
    parseOut = DESutils.parseOutfile(outfile)
    if parseOut.finished:
        times = np.append(times, parseOut.totalTime.value)
        GPtimes = np.append(GPtimes, parseOut.avgGPTime.value)
        nOpt1 = np.append(nOpt1, parseOut.nOpt1)
        nOpt2 = np.append(nOpt2, parseOut.nOpt2)
        nGP = np.append(nGP, parseOut.nGP)
    else:
        if parseOut.LinAlgErr:
            print(outfile)
            print(parseOut.LinAlgErr_params)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(GPtimes, bins=20, range=(0, 10))

percs = np.percentile(GPtimes, (25, 50, 75))
plt.axvline(x=np.mean(GPtimes), c="tab:red", linewidth=2, ls="-", label=f"{np.round(np.mean(GPtimes), 2)} mins (Mean)")
plt.axvline(x=percs[0], c="black", linewidth=2, ls=":", label=f"{np.round(percs[0], 2)} mins \n(25th Percentile)")
plt.axvline(x=percs[1], c="black", linewidth=2, ls="--", label=f"{np.round(percs[1], 2)} mins \n(50th Percentile)")
plt.axvline(x=percs[2], c="black", linewidth=2, ls=":", label=f"{np.round(percs[2], 2)} mins \n(75th Percentile)")
plt.legend(loc="upper right", framealpha=0.75)

plt.xlim((0, 10))
plt.ylim((0, None))

plt.xlabel("Mean GP Calculation Time [min]")
plt.ylabel("Number of Exposures [counts]")

plt.grid(which="minor", axis="x")
plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(times, bins=80, range=(0, 40))

percs = np.percentile(times, (25, 50, 75))
plt.axvline(x=np.mean(times), c="tab:red", linewidth=2, ls="-", label=f"{np.round(np.mean(times), 2)} hours (Mean)")
plt.axvline(x=percs[0], c="black", linewidth=2, ls=":", label=f"{np.round(percs[0], 2)} hours \n(25th Percentile)")
plt.axvline(x=percs[1], c="black", linewidth=2, ls="--", label=f"{np.round(percs[1], 2)} hours \n(50th Percentile)")
plt.axvline(x=percs[2], c="black", linewidth=2, ls=":", label=f"{np.round(percs[2], 2)} hours \n(75th Percentile)")
plt.legend(loc="upper right", framealpha=0.75)

plt.xlim((0, 24))
plt.ylim((0, None))

plt.xlabel("Total Time [hours]")
plt.ylabel("Number of Exposures [counts]")

plt.grid(which="minor", axis="x")
plt.grid()

plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True, figsize=(20, 10))
fig.subplots_adjust(wspace=0)

histrange = (0, 150)
nBins = 25

axes[0].axvline(x=np.mean(nOpt1), c="tab:red", linewidth=2, ls="-", label=f"{np.mean(nOpt1):.2f} Steps\nMean Opt1")
axes[0].hist(nOpt1, color="tab:red", histtype="stepfilled", range=histrange, bins=nBins)
axes[0].legend(loc="upper right")
axes[0].grid()

axes[1].axvline(x=np.mean(nOpt2), c="tab:blue", linewidth=2, ls="-", label=f"{np.mean(nOpt2):.2f} Steps\nMean Opt2")
axes[1].hist(nOpt2, color="tab:blue", histtype="stepfilled", range=histrange, bins=nBins)
axes[1].legend(loc="upper right")
axes[1].grid()

axes[2].axvline(x=np.mean(nOpt1+nOpt2), c="black", linewidth=2, ls="-", label=f"{np.mean(nOpt1+nOpt2):.2f} Steps\nMean Total")
axes[2].hist(nOpt1+nOpt2, color="black", histtype="stepfilled", range=histrange, bins=nBins)
axes[2].legend(loc="upper right")
axes[2].grid()
        

plt.show()